In [42]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys

https://data.gov.ie/dataset/a8e90f64-ddd1-4c29-97f4-dbabac4b0bee/resource/ada656d9-f9d4-4dfb-9049-62a3a526934a

In [43]:


df=pd.read_csv("hsa-incidents-2017-2021 extract.csv")
df.head()
#info for histograms

,Age,Gender,NACE Sector,Incident Type,Employment Status,Year,County,Trigger
0,0-17 Years,Male,F - CONSTRUCTION,Non-fatal injury,Employee,2017,Carlow,"Lifting, carrying"
1,0-17 Years,Female,C - MANUFACTURING,Non-fatal injury,Employee,2017,Dublin,Other triggers not listed
2,0-17 Years,Female,Q - HUMAN HEALTH AND SOCIAL WORK ACTIVITIES,Non-fatal injury,Employee,2017,Dublin,"Fall on same level (slip, stumble etc)"
3,0-17 Years,Male,C - MANUFACTURING,Non-fatal injury,Employee,2017,Offaly,"Pushing, pulling"
4,0-17 Years,Male,I - ACCOMMODATION AND FOOD SERVICE ACTIVITIES,Non-fatal injury,Employee,2017,Dublin,Loss of control of machine


In [44]:
sns.pairplot(df, hue= "Trigger")

year column is int 64 bit

In [45]:
df['Year']


0        2017
1        2017
2        2017
3        2017
4        2017
         ... 
45892    2021
45893    2021
45894    2021
45895    2021
45896    2021
Name: Year, Length: 45897, dtype: int64

adding a new varaible 

In [46]:
pd.to_datetime(df['Year'])

0       1970-01-01 00:00:00.000002017
1       1970-01-01 00:00:00.000002017
2       1970-01-01 00:00:00.000002017
3       1970-01-01 00:00:00.000002017
4       1970-01-01 00:00:00.000002017
                     ...             
45892   1970-01-01 00:00:00.000002021
45893   1970-01-01 00:00:00.000002021
45894   1970-01-01 00:00:00.000002021
45895   1970-01-01 00:00:00.000002021
45896   1970-01-01 00:00:00.000002021
Name: Year, Length: 45897, dtype: datetime64[ns]

new variable datetime added

In [53]:
df['datetime'] = pd.to_datetime(df['Year'], format = '%Y')
df

,Age,Gender,NACE Sector,Incident Type,Employment Status,Year,County,Trigger,datetime
0,0-17 Years,Male,F - CONSTRUCTION,Non-fatal injury,Employee,2017,Carlow,"Lifting, carrying",2017-01-01
1,0-17 Years,Female,C - MANUFACTURING,Non-fatal injury,Employee,2017,Dublin,Other triggers not listed,2017-01-01
2,0-17 Years,Female,Q - HUMAN HEALTH AND SOCIAL WORK ACTIVITIES,Non-fatal injury,Employee,2017,Dublin,"Fall on same level (slip, stumble etc)",2017-01-01
3,0-17 Years,Male,C - MANUFACTURING,Non-fatal injury,Employee,2017,Offaly,"Pushing, pulling",2017-01-01
4,0-17 Years,Male,I - ACCOMMODATION AND FOOD SERVICE ACTIVITIES,Non-fatal injury,Employee,2017,Dublin,Loss of control of machine,2017-01-01
...,...,...,...,...,...,...,...,...,...
45892,NaN,NaN,C - MANUFACTURING,Dangerous Occurrence,NaN,2021,Mayo,NaN,2021-01-01
45893,NaN,NaN,Q - HUMAN HEALTH AND SOCIAL WORK ACTIVITIES,Dangerous Occurrence,NaN,2021,Dublin,NaN,2021-01-01
45894,NaN,NaN,F - CONSTRUCTION,Dangerous Occurrence,NaN,2021,Dublin,NaN,2021-01-01
45895,NaN,NaN,Q - HUMAN HEALTH AND SOCIAL WORK ACTIVITIES,Dangerous Occurrence,NaN,2021,Dublin,NaN,2021-01-01
